In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Iterative Policy Evaluation in Grid-World**
- fix the state transfer prob matrix S
- fix the reward of each grid =-1
- policy is equiprobable

In [1]:
class GridWorld():
    
    def __init__(self,size,reward,itera):
        
        self.size = size
        self.iter = itera
        
        # instant reward, represents the first part of bellman equation
        # sum_{a}pi(a|s)*sum_{s',r}p(s',r|s,a)*r
        # r is set to be reward
        self.instant_r = reward*np.ones((self.size,self.size))
        self.instant_r[0,0]=0
        self.instant_r[-1,-1]=0
#         self.instant_r[0,1]=reward*0.75
#         self.instant_r[-1,-2]=reward*0.75
        
    def __DataStruct(self):
        
        self.policy=[0.25,0.25,0.25,0.25]
        
        # Initialize value-function matrix
        self.large_Vk = np.ones((self.size+2,self.size+2))*(-1)
        self.large_Vk[1,1]=0
        self.large_Vk[self.size,self.size]=0
        
        # execute-action related arrays
        self.actions_dict = {'up':(-1)*np.ones((self.size,self.size)),
                             'left':(-1)*np.ones((self.size,self.size)),
                             'down':np.ones((self.size,self.size)),
                             'right':np.ones((self.size,self.size))}
        
        self.cod = np.meshgrid(list(range(1,self.size+1)),
                               list(range(1,self.size+1)))

        self.move_left = (self.cod[0]+self.actions_dict['up']).astype(int)
        self.move_right = (self.cod[0]+self.actions_dict['down']).astype(int)
        self.move_up = (self.cod[1]+self.actions_dict['left']).astype(int)
        self.move_down = (self.cod[1]+self.actions_dict['right']).astype(int)
        
    def Act_Update(self):
        
        Vk_left = self.large_Vk[self.cod[1],self.move_left]
        Vk_right = self.large_Vk[self.cod[1],self.move_right]
        Vk_up = self.large_Vk[self.move_up,self.cod[0]]
        Vk_down = self.large_Vk[self.move_down,self.cod[0]]
        
        self.large_Vk[self.cod[1],self.cod[0]] = (0.25*Vk_up+0.25*Vk_down+0.25*Vk_left+0.25*Vk_right+self.instant_r)
        self.large_Vk[1,1]=0
        self.large_Vk[self.size,self.size]=0
        
        self.large_Vk[0,:] = self.large_Vk[1,:]
        self.large_Vk[:,-1] = self.large_Vk[:,-2]
        self.large_Vk[-1,:] = self.large_Vk[-2,:]
        self.large_Vk[:,0] = self.large_Vk[:,1]
        
    def ReadyPlayer1(self):
        
        self.__DataStruct()
        
        for _ in range(self.iter):
            
            self.Act_Update()